In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn
from scipy.stats import poisson,skellam

In [3]:
epl_1617 = pd.read_csv("http://www.football-data.co.uk/mmz4281/1617/E0.csv")
epl_1617 = epl_1617[['HomeTeam','AwayTeam','FTHG','FTAG']]
epl_1617 = epl_1617.rename(columns={'FTHG': 'HomeGoals', 'FTAG': 'AwayGoals'})
epl_1617.head()

,HomeTeam,AwayTeam,HomeGoals,AwayGoals
0,Burnley,Swansea,0,1
1,Crystal Palace,West Brom,0,1
2,Everton,Tottenham,1,1
3,Hull,Leicester,2,1
4,Man City,Sunderland,2,1


In [4]:
epl_1617.shape

(380, 4)

In [5]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

goal_model_data = pd.concat([epl_1617[['HomeTeam','AwayTeam','HomeGoals']].assign(home=1).rename(
            columns={'HomeTeam':'team', 'AwayTeam':'opponent','HomeGoals':'goals'}),
           epl_1617[['AwayTeam','HomeTeam','AwayGoals']].assign(home=0).rename(
            columns={'AwayTeam':'team', 'HomeTeam':'opponent','AwayGoals':'goals'})])

In [6]:
goal_model_data

,team,opponent,goals,home
0,Burnley,Swansea,0,1
1,Crystal Palace,West Brom,0,1
2,Everton,Tottenham,1,1
3,Hull,Leicester,2,1
4,Man City,Sunderland,2,1
...,...,...,...,...
375,Middlesbrough,Liverpool,0,0
376,Crystal Palace,Man United,0,0
377,Stoke,Southampton,1,0
378,West Brom,Swansea,1,0


In [7]:
goal_model_data_ = goal_model_data[:100]

In [8]:
poisson_model = smf.glm(formula="goals ~ home + team + opponent", data=goal_model_data_, 
                        family=sm.families.Poisson()).fit()
poisson_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                  goals   No. Observations:                  100
Model:                            GLM   Df Residuals:                       61
Model Family:                 Poisson   Df Model:                           38
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -134.35
Date:                Sat, 14 Aug 2021   Deviance:                       78.403
Time:                        15:25:34   Pearson chi2:                     65.3
No. Iterations:                    10                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept                      0.0950      0.353      0.269      0.788      -0.596       0.786
team[T.Bournemouth]           -0.1736      0.552     -0.314      0.753      -1.256       0.909
team[T.Burnley]               -0.7787      0.527     -1.478      0.139      -1.811       0.254
team[T.Chelsea]               -0.3130      0.482     -0.650      0.516      -1.257       0.631
team[T.Crystal Palace]        -0.7021      0.546     -1.286      0.199      -1.772       0.368
team[T.Everton]               -0.3591      0.549     -0.654      0.513      -1.436       0.718
team[T.Hull]                  -1.5749      0.702     -2.243      0.025      -2.951      -0.199
team[T.Leicester]             -0.4169      0.522     -0.799      0.424      -1.439       0.606
team[T.Liverpool]             -0.2174      0.530     -0.410      0.682      -1.257       0.822
team[T.Man City]              -0.0784      0.509     -0.154      0.878      -1.077       0.920
team[T.Man United]            -0.6617      0.541     -1.222      0.222      -1.723       0.399
team[T.Middlesbrough]         -0.9496      0.616     -1.541      0.123      -2.157       0.258
team[T.Southampton]           -0.8381      0.548     -1.529      0.126      -1.912       0.236
team[T.Stoke]                 -0.2907      0.547     -0.532      0.595      -1.362       0.781
team[T.Sunderland]            -0.5289      0.614     -0.861      0.389      -1.733       0.675
team[T.Swansea]               -1.2675      0.621     -2.042      0.041      -2.484      -0.051
team[T.Tottenham]             -0.9500      0.564     -1.685      0.092      -2.055       0.155
team[T.Watford]               -0.5678      0.532     -1.067      0.286      -1.611       0.475
team[T.West Brom]             -0.3856      0.585     -0.660      0.510      -1.531       0.760
team[T.West Ham]              -0.8165      0.600     -1.360      0.174      -1.993       0.360
opponent[T.Bournemouth]        1.0774      0.713      1.512      0.131      -0.319       2.474
opponent[T.Burnley]            1.1577      0.711      1.627      0.104      -0.237       2.552
opponent[T.Chelsea]            0.6904      0.750      0.920      0.357      -0.780       2.161
opponent[T.Crystal Palace]     0.8888      0.707      1.257      0.209      -0.497       2.274
opponent[T.Everton]            0.1686      0.765      0.220      0.826      -1.331       1.668
opponent[T.Hull]               1.2924      0.687      1.880      0.060      -0.055       2.640
opponent[T.Leicester]          1.4763      0.696      2.120      0.034       0.111       2.841
opponent[T.Liverpool]          0.9025      0.704      1.282      0.200      -0.477       2.282
opponent[T.Man City]           0.4594      0.791      0.581      0.561      -1.090       2.009
opponent[T.Man United]         0.7420      

In [11]:
predicted = poisson_model.predict(goal_model_data[101:])

In [ ]:
MSE = np.square(np.subtract(y_wo,predicted_wo)).mean()
print(MSE)

In [1]:
dfactual = goal_model_data[101:].reset_index()
dfactual.head()

NameError: name 'goal_model_data' is not defined

In [23]:
predicteddf = pd.DataFrame(predicted, columns = ['Pre'])
predicteddf.shape

(659, 1)

In [26]:
dfactual.to_csv(r"D:\Semester 7\Machine Learning\Assignment\dfactual.csv")
predicteddf.to_csv(r"D:\Semester 7\Machine Learning\Assignment\predicteddf.csv")

In [ ]:
df = pd.read_csv